In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [21]:
!pip install hdbscan

In [36]:
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import hdbscan
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import scipy.stats as stats
import joblib

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [17]:
player = pd.read_csv("/content/drive/MyDrive/2024 컨퍼/player_data.csv")

In [14]:
player.head()

,match_id,map_name,game_mode,player_id,player_name,player_account_id,primary_weapon,secondary_weapon,armor_type,use_of_health_items,use_of_boost_items,items_carried,time_spent_looting_sec,time_spent_in_combat_sec,kills,damage_dealt,movement_routes,first_location_x,first_location_y,first_location_z,final_location_x,final_location_y,final_location_z,walk_distance,swim_distance,ride_distance,road_kills,vehicle_destroys,weapons_acquired,boosts,heals,kill_streaks,headshot_kills,assists,revives,team_kills,win_place,team_id,team_rank,team_won,elapsedTime,numAlivePlayers
0,02c7c736-e103-4c2c-a8e7-ea4c967c379d,Savage_Main,squad,2727aabd-287a-41dc-adce-04148d7cdafc,seulO_O,account.ae62cf516f08404ab2a7f13130efa57a,Item_Weapon_MP5K_C,Item_Weapon_Pan_C,NaN,0,0,"Item_Back_B_01_StartParachutePack_C, Item_Heal...",1158.429,15.100,0,62.616158,"(116174.3125,164796.140625,15883.021484375) ->...",116174.312500,164796.140625,15883.021484,193899.578125,189967.656250,1023.330017,2989.499000,0.0,0.0,0,0,6,5,1,0,0,1,0,0,4,10.0,4.0,False,0.0,80.0
1,02c7c736-e103-4c2c-a8e7-ea4c967c379d,Savage_Main,squad,4f3cd113-9a48-43c3-82f3-f9045cfb21a8,H_O_L_I_C,account.fd5033f4029d496b8c672108f191efb3,Item_Weapon_Thompson_C,Item_Weapon_MP5K_C,NaN,0,0,"Item_Back_B_01_StartParachutePack_C, Item_Ammo...",236.786,2.612,0,0.000000,"(116750,164678.5,15888.599609375) -> (116750,1...",116750.000000,164678.500000,15888.599609,341549.625000,157000.656250,734.809998,223.724430,0.0,0.0,0,0,5,0,0,0,0,0,0,0,17,26.0,17.0,False,0.0,95.0
2,02c7c736-e103-4c2c-a8e7-ea4c967c379d,Savage_Main,squad,fbd5d5b1-e3e8-49c3-8e5f-8b03a607ca56,ZER0_100,account.73ab78e3325d4cfbb9f2101683736830,Item_Weapon_ACE32_C,Item_Weapon_Winchester_C,NaN,0,0,"Item_Back_B_01_StartParachutePack_C, Item_Weap...",187.917,245.581,0,100.000010,"(251288.671875,137562.703125,3846.568603515625...",251288.671875,137562.703125,3846.568604,241000.921875,345338.250000,1709.209961,120.128624,0.0,0.0,0,0,4,0,0,0,0,0,0,0,21,7.0,21.0,False,0.0,78.0
3,02c7c736-e103-4c2c-a8e7-ea4c967c379d,Savage_Main,squad,e87810c3-6195-49ca-995a-672b13a23826,Anthony-Davis2,account.dc5cce3eacc74887b2640825394a4cfc,Item_Weapon_HK416_C,NaN,NaN,0,0,"Item_Back_B_01_StartParachutePack_C, Item_Weap...",72.506,5.302,1,122.800000,"(115997.703125,164408.40625,15888.599609375) -...",115997.703125,164408.406250,15888.599609,194661.078125,197629.046875,1016.320007,27.147842,0.0,0.0,0,0,1,0,0,1,1,0,0,0,23,5.0,23.0,False,0.0,82.0
4,02c7c736-e103-4c2c-a8e7-ea4c967c379d,Savage_Main,squad,e766fb44-d306-43f0-9983-3bc6acb92a29,ksj2837,account.20dc2e5c10d7426fbd8f24dbb948cb4c,Item_Weapon_BerylM762_C,Item_Weapon_M79_C,NaN,0,0,"Item_Back_B_01_StartParachutePack_C, Item_Back...",761.401,1426.494,2,299.073330,"(115515.4296875,164927.125,15878.927734375) ->...",115515.429688,164927.125000,15878.927734,189028.125000,178672.000000,638.890015,3896.207800,0.0,0.0,0,0,13,5,1,1,0,4,1,0,1,6.0,1.0,True,0.0,82.0


전처리

In [18]:
def preprocess_player_data(df):

    # 스쿼드 모드만 추출
    df = df[df['game_mode'] == 'squad']

    df = df[['match_id', 'player_id', 'player_account_id', 'team_rank','assists', 'revives', 'first_location_x','first_location_y', 'first_location_z', 'movement_routes']]

    # 중복 플레이어 제거
    df = df[df.duplicated(subset=['player_id', 'match_id'], keep=False) == False]

    # team_id 생성: match_id와 team_rank 결합
    df['team_id'] = df['match_id'].astype(str) + "_" + df['team_rank'].astype(str)

    # team_id가 같은 데이터들의 개수 확인
    team_id_counts = df.groupby('team_id').size().reset_index(name='count')

    # 각 고유 값이 몇 개씩 존재하는지 확인
    unique_value_counts = team_id_counts['count'].value_counts()

    # count 열이 4인 행의 team_id 값을 리스트로 가져오기
    id_values = team_id_counts.loc[team_id_counts['count'] == 4, 'team_id'].tolist()

    # team_id가 id_values에 속하는 행만 추출 (한 팀에 4명인 경우만 추출)
    df = df[df['team_id'].isin(id_values)]

    # 결측치 삭제
    df = df.dropna()

    return df

In [19]:
player = preprocess_player_data(player)
print(player.shape)

(79248, 11)


파생변수

In [24]:
def create_derived_features(df):

    # 팀 평균 위치 계산
    df['team_avg_x'] = df.groupby('team_id')['first_location_x'].transform('mean')
    df['team_avg_y'] = df.groupby('team_id')['first_location_y'].transform('mean')
    df['team_avg_z'] = df.groupby('team_id')['first_location_z'].transform('mean')

    # 팀 평균 위치와 개별 플레이어 위치 간 거리 계산
    df['first_location_to_team'] = np.sqrt(
        (df['first_location_x'] - df['team_avg_x'])**2 +
        (df['first_location_y'] - df['team_avg_y'])**2 +
        (df['first_location_z'] - df['team_avg_z'])**2
    )

    # 1. movement_routes 열에서 X, Y, Z 값 추출
    def extract_coordinates(movement_route):
        # 좌표 패턴을 찾기 위해 정규식을 사용
        pattern = r'\(([-\d\.]+),\s*([-\d\.]+),\s*([-\d\.]+)\)'

        # 정규식으로 X, Y, Z 좌표 리스트 추출
        coordinates = re.findall(pattern, movement_route)

        # 각 좌표를 X, Y, Z 리스트로 변환
        x_coords = [float(x) for x, _, _ in coordinates]
        y_coords = [float(y) for _, y, _ in coordinates]
        z_coords = [float(z) for _, _, z in coordinates]

        return x_coords, y_coords, z_coords

    # X, Y, Z 좌표 추출하여 새로운 열로 저장
    df[['x_coords', 'y_coords', 'z_coords']] = df['movement_routes'].apply(lambda route: pd.Series(extract_coordinates(route)))

    # 거리 계산 함수: 두 좌표 간의 유클리드 거리
    def euclidean_distance(player_pos, team_mean_pos):
        return np.sqrt(np.sum((player_pos - team_mean_pos) ** 2))

    # 각 팀별 거리 계산 함수
    def calculate_team_distances(team_df):
        # 팀원 중 가장 짧은 리스트 길이 구하기
        min_length = team_df['x_coords'].apply(len).min()

        # 팀 평균 위치 계산
        team_mean_positions = []
        for i in range(min_length):
            mean_x = np.mean([x[i] for x in team_df['x_coords']])
            mean_y = np.mean([y[i] for y in team_df['y_coords']])
            mean_z = np.mean([z[i] for z in team_df['z_coords']])
            team_mean_positions.append([mean_x, mean_y, mean_z])
        team_mean_positions = np.array(team_mean_positions)

        # 각 플레이어와 팀 평균 위치 간 거리 계산
        distances = []
        for _, row in team_df.iterrows():
            player_positions = np.array([[row['x_coords'][i], row['y_coords'][i], row['z_coords'][i]] for i in range(min_length)])
            distance = np.mean([euclidean_distance(player_pos, team_mean_positions[i]) for i, player_pos in enumerate(player_positions)]) / min_length
            distances.append(distance)

        return distances

    # 팀별 거리 계산 및 결과 추가
    def calculate_distances_with_group(df):
        distances = calculate_team_distances(df)
        df['location_to_team'] = distances  # 계산된 거리를 열로 추가
        return df

    # 그룹별로 거리 계산 수행
    df = df.groupby('team_id', group_keys=False).apply(calculate_distances_with_group)

    return df

In [25]:
player = create_derived_features(player)
print(player.shape)

(79248, 19)


스케일링

In [27]:
def scale_features(df, columns):

    # 로그 변환
    for col in columns:
        df[col] = np.log1p(df[col])  # log(1 + x) 변환

    # MinMax Scaling 적용
    scaler = MinMaxScaler()
    df[columns] = scaler.fit_transform(df[columns])

    return df


In [28]:
selected_columns = ['assists', 'revives', 'first_location_to_team', 'location_to_team']
player = scale_features(player, selected_columns)

라벨링

In [29]:
def apply_clustering(df, columns, kmeans_clusters=2, hdbscan_min_cluster_size=5000, hdbscan_min_samples=300):

    # 변수 선택
    X = df[columns]

    # KMeans 클러스터링 수행
    kmeans = KMeans(n_clusters=kmeans_clusters, random_state=42)
    df['km_cluster'] = kmeans.fit_predict(X)

    # HDBSCAN 클러스터링 수행
    clusterer = hdbscan.HDBSCAN(min_cluster_size=hdbscan_min_cluster_size, min_samples=hdbscan_min_samples)
    df['db_cluster'] = clusterer.fit_predict(X)

    # HDBSCAN 클러스터링 결과 변환
    # 1인 값은 임시 값 -2로 변경
    df.loc[df['db_cluster'] == 1, 'db_cluster'] = -2

    # 1이 아닌 값을 1(팀)으로 변경
    df.loc[df['db_cluster'] != -2, 'db_cluster'] = 1

    # 임시 값 -2을 0(개인)으로 변경
    df.loc[df['db_cluster'] == -2, 'db_cluster'] = 0

    # KMeans와 HDBSCAN 결과가 같은 데이터만 선택
    df = df[df['db_cluster'] == df['km_cluster']]

    return df


In [30]:
player = apply_clustering(player, selected_columns)

In [33]:
print(player.shape)
print(player[['db_cluster', 'km_cluster']].head())

(69457, 21)
   db_cluster  km_cluster
1           0           0
2           0           0
3           0           0
4           1           1
5           1           1


모델링

In [34]:
# X (입력 변수)와 y (타겟 변수) 분리
X = player[selected_columns]
y = player['db_cluster']

# 데이터 나누기: 80% 학습용, 20% 테스트용
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 선형 SVM 모델 생성 및 학습
linear_svm = SVC(kernel='linear', random_state=42)
linear_svm.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = linear_svm.predict(X_test)


In [37]:
joblib.dump(linear_svm, 'group_model_svm.pkl')

['group_model_svm.pkl']

가설 검정 (맨-휘트니검정)

In [35]:
X_test['y_pred'] = y_pred

non_team = X_test[X_test['y_pred'] == 0]
team = X_test[X_test['y_pred'] == 1]

In [39]:
# 'assists' 변수 추출
assists_0 = non_team['assists']
assists_1 = team['assists']

# 맨-휘트니 검정 수행
stat, p_value = stats.mannwhitneyu(assists_0, assists_1, alternative='two-sided')

# 결과 출력
print("Mann-Whitney U test statistic:", stat)
print("p-value:", p_value)

# p-value에 따른 결과 해석
if p_value < 0.05:
    print("두 집단 간 'assists'에 차이가 있습니다.")
else:
    print("두 집단 간 'assists'에 차이가 없습니다.")

Mann-Whitney U test statistic: 8444188.5
p-value: 0.0
두 집단 간 'assists'에 차이가 있습니다.


In [40]:
# 'revives' 변수 추출
revives_0 = non_team['revives']
revives_1 = team['revives']

# 맨-휘트니 검정 수행
stat, p_value = stats.mannwhitneyu(revives_0, revives_1, alternative='two-sided')

# 결과 출력
print("Mann-Whitney U test statistic:", stat)
print("p-value:", p_value)

# p-value에 따른 결과 해석
if p_value < 0.05:
    print("두 집단 간 'revives'에 차이가 있습니다.")
else:
    print("두 집단 간 'revives'에 차이가 없습니다.")

Mann-Whitney U test statistic: 3677553.0
p-value: 0.0
두 집단 간 'revives'에 차이가 있습니다.


In [41]:
#'first_location_to_team' 변수 추출
first_location_to_team_0 = non_team['first_location_to_team']
first_location_to_team_1 = team['first_location_to_team']

# 맨-휘트니 검정 수행
stat, p_value = stats.mannwhitneyu(first_location_to_team_0, first_location_to_team_1, alternative='two-sided')

# 결과 출력
print("Mann-Whitney U test statistic:", stat)
print("p-value:", p_value)

# p-value에 따른 결과 해석
if p_value < 0.05:
    print("두 집단 간 'first_location_to_team'에 차이가 있습니다.")
else:
    print("두 집단 간 'first_location_to_team'에 차이가 없습니다.")

Mann-Whitney U test statistic: 20043956.0
p-value: 2.289892086297739e-06
두 집단 간 'first_location_to_team'에 차이가 있습니다.


In [42]:
# 'location_to_team' 변수 추출
location_to_team_0 = non_team['location_to_team']
location_to_team_1 = team['location_to_team']

# 맨-휘트니 검정 수행
stat, p_value = stats.mannwhitneyu(location_to_team_0, location_to_team_1, alternative='two-sided')

# 결과 출력
print("Mann-Whitney U test statistic:", stat)
print("p-value:", p_value)

# p-value에 따른 결과 해석
if p_value < 0.05:
    print("두 집단 간 'location_to_team'에 차이가 있습니다.")
else:
    print("두 집단 간 'location_to_team'에 차이가 없습니다.")

Mann-Whitney U test statistic: 28221515.0
p-value: 0.0
두 집단 간 'location_to_team'에 차이가 있습니다.
